In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("../../src/juliaEAM.jl")
include("../../src/lammpsIO.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using Unitful: Å, nm
using StaticArrays: SVector
using Molly
using LinearAlgebra
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable
using ProgressMeter
using Random

  Activating new project at `~/Documents/ABCDE/ABCD_J/EAM/surface`


## Define Copper

In [2]:
# include("../lammps_wrap/juliaEAM.jl")

Cu_LatConst = 3.615/10 # nm
atom_mass = 63.546u"u"  # Atomic mass of aluminum in grams per mole

## `AtomCalculators` force/energy calculator and `Molly` simulator
eam = EAM()
fname = "Cu_Zhou04.eam.alloy"
read_potential!(eam, fname)

### Define customized interaction type in `AtomsCalculators`
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
    f_atomstress::Any  # Holds the atomic level stresses function
    pair_coeff_string::String  # Holds the pair coefficient string
end

## Initialize sys

In [3]:
function initialize_system_dump(;loggers=(coords=CoordinateLogger(1),),filename_dump="")
    n_atoms, box_size, coords_molly, attypes = lmpDumpReader(filename_dump)
    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                    #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                    ) for i in 1:length(coords_molly)]
    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(coords_molly)
    molly_atoms_init = copy(molly_atoms)

    DNF = DistanceNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
        n_steps=1e3,
        dist_cutoff=7u"Å")
    TNF = TreeNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)), 
        n_steps=1e3,
        dist_cutoff=7u"Å")

    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Cu", atom_type=string(attypes[ia])) for (ia,a) in enumerate(molly_atoms_init)],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DNF,
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system_dump (generic function with 1 method)

In [4]:
filename_dump = "./out_surface.dump"
molly_system = initialize_system_dump(filename_dump = filename_dump)
neighbors_all = get_neighbors_all(molly_system)

7180-element Vector{Vector{Int64}}:
 [25, 1021, 1045, 1729, 1753, 2500, 2538, 2539, 2540, 2542  …  6511, 6551, 6553, 6555, 6557, 6751, 6791, 6792, 6795, 6796]
 [3, 4, 5, 6, 7, 8, 9, 12, 45, 46  …  3876, 3877, 3878, 4119, 4120, 4122, 4123, 4124, 4126, 4164]
 [2, 4, 5, 6, 7, 8, 9, 12, 45, 46  …  4130, 4163, 4164, 4165, 4166, 4407, 4408, 4411, 4412, 4452]
 [2, 3, 5, 6, 7, 8, 42, 43, 45, 46  …  4123, 4124, 4126, 4160, 4163, 4164, 4165, 4166, 4408, 4452]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12  …  4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4130, 4164]
 [2, 3, 4, 5, 7, 8, 9, 10, 11, 12  …  3841, 3842, 3846, 4120, 4123, 4124, 4126, 4127, 4128, 4130]
 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12  …  4128, 4129, 4130, 4131, 4134, 4408, 4411, 4412, 4415, 4416]
 [2, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  4123, 4124, 4125, 4126, 4127, 4128, 4130, 4164, 4408, 4412]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4134]
 [5, 6, 7, 8, 9, 11, 12, 13, 14, 15  …  3845, 3846, 385

In [5]:
include("../../src/simulator.jl")
function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
    n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 
    
    fs = interaction.f_forces(sys, interaction.pair_coeff_string)

    # Add penalty term to forces
    if penalty_coords != nothing
        fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
        # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
    end
    return fs
end

In [6]:
eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces_LAMMPS,calculate_atomstress,"pair_coeff * * Cu_Zhou04.eam.alloy Cu")

EAMInteractionJulia(EAM(1, ["Cu"], 2000, 0.05002501250625312, 2000, 0.002859305649575697, 5.715751993501819, [0.0 -0.01357527207849252 … -1.834392661415103 -1.834127593569618], [50.80361252664051 50.80361252664051 … 2.954726881253157e-6 2.896308622055337e-6], [29], [63.546], [3.615], "fcc", [128.0308779828199;;; 128.0308779828199;;; 128.0308779828199;;; … ;;; -6.068483307636601e-6;;; -5.95141389855619e-6;;; -5.836667719215691e-6], 0.0:0.002859305649575697:5.715751993501819, 0.0:0.05002501250625312:100.0, [0.0, 0.002859305649575697, 0.005718611299151394, 0.008577916948727091, 0.011437222598302788, 0.014296528247878484, 0.017155833897454183, 0.020015139547029877, 0.022874445196605576, 0.025733750846181274  …  5.690018242655637, 5.692877548305213, 5.695736853954789, 5.698596159604364, 5.7014554652539395, 5.704314770903515, 5.707174076553091, 5.710033382202667, 5.712892687852243, 5.715751993501819], [0.0, 0.05002501250625312, 0.10005002501250625, 0.1500750375187594, 0.2001000500250125, 0.2

energy minimization

In [ ]:
# frozen_atoms = []
# z_coords = [coords[3] for coords in molly_system.coords]
# frozen_atoms = [index for (index, z) in enumerate(z_coords) if z ≤ 1u"Å"]

# simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=20, step_size_minimize=5e-3u"ps", tol=1e-6u"eV/Å")
# Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
#          n_threads=1, frozen_atoms=frozen_atoms, neig_interval=5, print_nsteps=true,
#          mass=atom_mass)
# neighbors_all = get_neighbors_all(molly_system)

7180-element Vector{Vector{Int64}}:
 [25, 1021, 1045, 1729, 1753, 2500, 2538, 2539, 2540, 2542  …  6511, 6551, 6553, 6555, 6557, 6751, 6791, 6792, 6795, 6796]
 [3, 4, 5, 6, 7, 8, 9, 12, 45, 46  …  3876, 3877, 3878, 4119, 4120, 4122, 4123, 4124, 4126, 4164]
 [2, 4, 5, 6, 7, 8, 9, 12, 45, 46  …  4130, 4163, 4164, 4165, 4166, 4407, 4408, 4411, 4412, 4452]
 [2, 3, 5, 6, 7, 8, 42, 43, 45, 46  …  4123, 4124, 4126, 4160, 4163, 4164, 4165, 4166, 4408, 4452]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12  …  4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4130, 4164]
 [2, 3, 4, 5, 7, 8, 9, 10, 11, 12  …  3841, 3842, 3846, 4120, 4123, 4124, 4126, 4127, 4128, 4130]
 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12  …  4128, 4129, 4130, 4131, 4134, 4408, 4411, 4412, 4415, 4416]
 [2, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  4123, 4124, 4125, 4126, 4127, 4128, 4130, 4164, 4408, 4412]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4134]
 [5, 6, 7, 8, 9, 11, 12, 13, 14, 15  …  3845, 3846, 385

run ABC

In [8]:
# frozen_atoms = []
z_coords = [coords[3] for coords in molly_system.coords]
frozen_atoms = [index for (index, z) in enumerate(z_coords) if z ≤ 1u"Å"]

nopenalty_atoms = []
N_free = length(molly_system.coords)-length(nopenalty_atoms)
print("exclude ",length(nopenalty_atoms)," atoms from E_phi calculation\n")
print("using ",N_free," atoms for E_phi calculation\n")

# sigma = sqrt(0.006*3*N_free)
sigma = sqrt(0.2)
W = 0.1
@printf("sigma^2 = %e, %e Å/dof^1/2\n W = %e eV\n",ustrip(sigma^2), ustrip(sigma/sqrt(3*N_free)),ustrip(W))

simulator = ABCSimulator(sigma=sigma*u" Å", W=W*u"eV", 
                         max_steps=100, max_steps_minimize=30, step_size_minimize=5e-3u"ps", tol=1e-3u"eV/Å")

simulate!(molly_system, simulator, eamJulia, n_threads=1, 
         #   fname="output_stress_cube8.txt", fname_dump="stress_cube8.dump", fname_min_dump="min_stress_cube8.dump",
         fname="test.txt", fname_dump="test.dump", fname_min_dump="test.dump", # for speed test
         neig_interval=30, loggers_interval=10, dump_interval=10, start_dump=0,
         minimize_only=false, 
         d_boost=1e-6u"Å", 
         frozen_atoms=frozen_atoms, nopenalty_atoms=nopenalty_atoms, 
         p_drop = 1-1/32, p_keep=0, n_memory=0, n_search=100,
         p_stress = 1-192/7180, n_stress=12)

exclude 0 atoms from E_phi calculation
using 7180 atoms for E_phi calculation
sigma^2 = 2.000000e-01, 3.047138e-03 Å/dof^1/2
 W = 1.000000e-01 eV
step 0: -25030.681347432528 eV


Progress: 100%|█████████████████████████████████████████| Time: 0:06:31


System with 7180 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[43.290503414318835 Å, 43.290503414318835 Å, 72.15083902386472 Å])